In [1]:
%pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from tensorflow.keras.callbacks import EarlyStopping

UsageError: Line magic function `%kagglehub` not found.


In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
cnn = Sequential()
# First convolutional block
cnn.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[128, 128, 3]))
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Second convolutional block
cnn.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Third convolutional block
cnn.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Fourth convolutional block
cnn.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2)) 

# Fifth convolutional block
cnn.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
cnn.add(Conv2D(filters=512, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Dropout layer
cnn.add(Dropout(0.25))

# Flatten the output
cnn.add(Flatten())

# Dense layers
cnn.add(Dense(units=1500, activation='relu'))
cnn.add(Dropout(0.4))  # To avoid overfitting
cnn.add(Dense(units=38, activation='softmax'))

cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
             loss='categorical_crossentropy', 
             metrics=['accuracy'])

cnn.summary()

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
history = cnn.fit(x = training_set, validation_data = validation_set,epochs=30)

In [ ]:
train_loss, train_acc = cnn.evaluate(training_set)
print("Training Accuracy: ", train_acc)
print("Training loss: ", train_loss)

In [ ]:
val_loss, val_acc = cnn.evaluate(validation_set)
print("Validation Accuracy: ", val_acc)
print("Validation loss: ", val_loss)

In [ ]:
import json
with open('class_names.json', 'w') as f:
    json.dump(training_set.class_names, f)

# now save your trained model
cnn.save("plant-disease-detection-model.keras")


In [ ]:
# Get training and validation accuracy/loss
history_dict = history.history
train_loss = history_dict['loss']
val_loss = history_dict['val_loss']
train_acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# Create epochs range
epochs = range(1, len(train_loss) + 1)

# Plot Training & Validation Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')  # 'bo-' -> blue dots with line
plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')   # 'r*-' -> red stars with line
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()

# Plot Training & Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_acc, 'bo-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r*-', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()

plt.show()

In [ ]:
class_name = validation_set.class_names
print(class_name)

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
predicted_categories = tf.argmax(y_pred, axis = 1)

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
y_true = tf.argmax(true_categories, axis=1)

In [ ]:
y_true

In [ ]:
predicted_categories

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_true, predicted_categories)

print(classification_report(y_true, predicted_categories, target_names=class_name))

In [ ]:
cnn_model = tf.keras.models.load_model('/kaggle/working/plant-disease-detection-model.keras')

In [ ]:
import cv2
img_path = '/kaggle/input/test/test/AppleCedarRust1.JPG'

img = cv2.imread(img_path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)


plt.imshow(img)
plt.title('Test Image :AppleCedarRust')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
image = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr]) # Convert single image to a batch
predictions = cnn_model.predict(input_arr)

In [ ]:
result_index = np.argmax(predictions) 
print(result_index)

In [ ]:
# Displaying the disease prediction
model_prediction = class_name[result_index]
plt.imshow(img) 
plt.title(f"Disease Name: {model_prediction}")

plt.xticks([])
plt.yticks([])

plt.show()